In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix
from sklearn.pipeline import make_pipeline

## Load Data
Load the training and testing data.

In [ ]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

X_train = train.drop('Survived', axis=1).copy()
y_train = train['Survived'].copy()
X_test = test.copy()

X_train.Age.fillna(round(X_train.Age.mean()), inplace=True)
X_test.Age.fillna(round(X_test.Age.mean()), inplace=True)
X_test.Fare.fillna(X_test.Fare.mean(), inplace=True)

## Clean up
Clean up the data to contain only the relevant data used for prediction, and one-hot-encode all the categorical data.

In [ ]:
irrelevant_columns = ['Ticket', 'Cabin', 'Name', 'PassengerId']
categorical_columns = ['Sex', 'Pclass', 'Embarked']
X_train_encoded = pd.get_dummies(X_train.drop(irrelevant_columns, axis=1).copy(), columns=categorical_columns)
X_test_encoded = pd.get_dummies(X_test.drop(irrelevant_columns, axis=1).copy(), columns=categorical_columns)

In [ ]:
X_train_scaled = scale(X_train_encoded)
X_test_scaled = scale(X_test_encoded)
np.isnan(X_test_scaled).any()
    

In [ ]:
clf_svm = SVC(random_state=76)
clf_svm.fit(X_train_scaled, y_train)

In [ ]:
param_grid = [{
    'C': [0.5, 1, 10, 100],
    'gamma': ['scale', 1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf']
}]

optimal_params = GridSearchCV(SVC(),
                             param_grid,
                             cv=5,
                             scoring='accuracy',
                             verbose=0)

optimal_params.fit(X_train_scaled, y_train)
print(optimal_params.best_params_)

In [ ]:
clf_svm = SVC(random_state=76, C=1, gamma=0.1)
clf_svm.fit(X_train_scaled, y_train)

In [ ]:
plot_confusion_matrix(clf_svm, 
                     X_train_scaled,
                     y_train,
                     display_labels=['Died', 'Survived'])

In [ ]:
predictions = pd.DataFrame()
predictions['PassengerId'] = pd.Series(X_test.PassengerId)
predictions['Survived'] = pd.Series(clf_svm.predict(X_test_scaled))

In [ ]:
predictions.to_csv('submission.csv', index=False)